# 1. Weather API
https://open-meteo.com/?utm_source=chatgpt.com

In [1]:
import requests

# Tọa độ của Hà Nội
latitude = 21.0285
longitude = 105.8542

# URL API
url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"

# Gửi yêu cầu GET đến API
response = requests.get(url)

# Kiểm tra nếu yêu cầu thành công
if response.status_code == 200:
    data = response.json()
    current_weather = data.get("current_weather", {})
    temperature = current_weather.get("temperature")
    windspeed = current_weather.get("windspeed")
    weather_code = current_weather.get("weathercode")
    time = current_weather.get("time")

    print(f"Thời gian: {time}")
    print(f"Nhiệt độ hiện tại: {temperature}°C")
    print(f"Tốc độ gió: {windspeed} km/h")
    print(f"Mã thời tiết: {weather_code}")
else:
    print(f"Lỗi khi lấy dữ liệu: {response.status_code}")


Thời gian: 2025-05-12T05:30
Nhiệt độ hiện tại: 28.2°C
Tốc độ gió: 3.9 km/h
Mã thời tiết: 2


Lat Long Thành phố

In [3]:
!pip install geopy

  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)


In [5]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="vn_location_app")
location = geolocator.geocode("Thái Bình, Việt Nam")

if location:
    print("Latitude:", location.latitude)
    print("Longitude:", location.longitude)
else:
    print("Không tìm thấy địa điểm.")


Latitude: 20.449288
Longitude: 106.3424967


# 2. Lấy data tới ngày hiện tại


In [9]:
import pandas as pd
import requests

provinces = {
    "Hà Nội": (21.0285, 105.8542),
    "Hồ Chí Minh": (10.762622, 106.660172),
    "Đà Nẵng": (16.047079, 108.206230),
    "Hải Phòng": (20.844912, 106.688085),
    "Cần Thơ": (10.045162, 105.746857)
}

# event_dates = ["2025-05-01", "2025-05-02"]
start_date = "2025-05-10"
today = pd.to_datetime("today").date()
event_dates = pd.date_range(start=start_date, end=today).strftime('%Y-%m-%d').tolist()

results = []

for date in event_dates:
    for province, (lat, lon) in provinces.items():
        url = (
            f"https://api.open-meteo.com/v1/forecast?"
            f"latitude={lat}&longitude={lon}&start_date={date}&end_date={date}"
            f"&daily=temperature_2m_max,temperature_2m_min,windspeed_10m_max,weathercode"
            f"&timezone=auto"
        )
        r = requests.get(url)
        if r.ok:
            data = r.json().get("daily", {})
            if data:
                results.append({
                    "province": province,
                    "date": date,
                    "temp_max": data["temperature_2m_max"][0],
                    "temp_min": data["temperature_2m_min"][0],
                    "windspeed_max": data["windspeed_10m_max"][0],
                    "weathercode": data["weathercode"][0]
                })

df = pd.DataFrame(results)
print(df)



       province        date  temp_max  temp_min  windspeed_max  weathercode
0        Hà Nội  2025-05-10      30.6      25.4           17.9           96
1   Hồ Chí Minh  2025-05-10      32.2      27.0            7.3           95
2       Đà Nẵng  2025-05-10      31.2      27.5           14.3           80
3     Hải Phòng  2025-05-10      29.9      26.4           12.6           96
4       Cần Thơ  2025-05-10      33.1      26.8           11.1           95
5        Hà Nội  2025-05-11      27.1      20.4           15.5           81
6   Hồ Chí Minh  2025-05-11      32.7      26.3            5.8            3
7       Đà Nẵng  2025-05-11      29.8      26.2           18.5           95
8     Hải Phòng  2025-05-11      26.6      20.4           17.1           80
9       Cần Thơ  2025-05-11      32.8      25.7            8.0           95
10       Hà Nội  2025-05-12      29.0      20.9            8.9            3
11  Hồ Chí Minh  2025-05-12      32.7      26.2           11.4           95
12      Đà N